**Import required libraries and scripts**

In [1]:
import os
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.performance_calculation import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *


[11:32:17] Initializing Normalizer
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/tony/.conda/envs/dockm8/lib/python3.8/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):


In [ ]:
for dir in os.listdir('/home/tony/DEKOIS-noprot'):
    print(dir)
    software = '/home/tony/DockM8/software'
    protein_file = f'/home/tony/DEKOIS-noprot/{dir}/receptor_protoss_prepared.pdb'
    ref_file = f'/home/tony/DEKOIS-noprot/{dir}/crystal_ligand_protoss.sdf'
    pocket = 'reference'
    protonation = 'pkasolver'
    docking_library = f'/home/tony/DEKOIS-noprot/{dir}/merged_actives_decoys.sdf'
    docking_programs = ['GNINA', 'SMINA', 'PLANTS']
    clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS']
    clustering_method = 'KMedoids'
    rescoring= ['gnina', 'AD4', 'chemplp', 'rfscorevs', 'LinF9', 'SCORCH', 'RTMScore', 'vinardo']
    id_column = 'ID'
    n_poses = 10
    exhaustiveness = 8
    parallel = 1
    ncpus = int(os.cpu_count()-2)
    #Create a temporary folder for all further calculations
    w_dir = os.path.dirname(protein_file)
    print('The working directory has been set to:', w_dir)
    create_temp_folder(w_dir+'/temp')
    
    try:
        apply_consensus_methods_combinations(w_dir, docking_library, clustering_metrics)
    except Exception as e:
        printlog(f'Failed for {dir}')
        print(e)
        


In [ ]:
def rename_columns_in_csv(root_dir, target_file, col_rename_dict):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if target_file in filename:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath)
                df.rename(columns=col_rename_dict, inplace=True)
                df.to_csv(filepath, index=False)
                print(f"Columns renamed in: {filepath}")

root_directory = "/home/tony/LecB/LecB-data.sdf"
target_csv_file = "allposes_rescored.csv"
column_rename_dict = {
    "AD4_Affinity": "AD4",
    "Vinardo_Affinity": "Vinardo",
    "GNINA_Affinity": "GNINA",
    "GNINA_CNN_Score": "CNN-Score",
    "GNINA_CNN_Affinity": "CNN-Affinity",
    'LinF9_Affinity':'LinF9',
    'SCORCH_pose_score':'SCORCH',
    # Add more column names to be renamed here
}
rename_columns_in_csv(root_directory, target_csv_file, column_rename_dict)

In [ ]:
import os
import pandas as pd

def rename_columns_in_csv(root_dir, target_file):
    first_file_processed = False
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if target_file in filename:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath)
                
                if not first_file_processed:
                    merged_df = df
                    first_file_processed = True
                else:
                    merged_df = pd.merge(merged_df, df, on='ID')
    
    # Save the merged dataframe to a new file
    merged_file_path = os.path.join(root_dir, 'merged.csv')
    merged_df.to_csv(merged_file_path, index=False)
    print(f"Merged data saved to: {merged_file_path}")

root_directory = "/home/tony/LecB/temp/analysis/"
target_csv_file = ".csv"

rename_columns_in_csv(root_directory, target_csv_file)


In [ ]:
import os
import pandas as pd

def rename_columns_in_csv(root_dir, target_file, col_rename_dict, column_suffix):
    merged_df = pd.DataFrame()
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if target_file in filename:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath)
                merged_df = pd.concat([merged_df, df], ignore_index=True)
    
    # Save the merged dataframe to a new file
    merged_file_path = os.path.join(root_dir, 'merged.csv')
    merged_df.to_csv(merged_file_path, index=False)
    print(f"Merged data saved to: {merged_file_path}")

root_directory = "/home/tony/LecB/temp/analysis/"
target_csv_file = ".csv"
column_rename_dict = {}  # Empty dictionary, no renaming
column_suffix = "_suffix"  # Add your desired suffix here

rename_columns_in_csv(root_directory, target_csv_file, column_rename_dict, column_suffix)


In [ ]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)
                dirname = os.path.basename(dirpath)
                print(dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', ''))
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', '')}, inplace=True)
                df.drop(columns='EF10%', inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["Scoring Function", "Clustering Metric"])

    if merged_df is not None:
        # Adding the average column
        columns_to_exclude = ['method_name', 'selected_columns', 'clustering_metric']
        numeric_columns = [col for col in merged_df.columns if col not in columns_to_exclude]
        merged_df['Average'] = merged_df[numeric_columns].mean(axis=1)
        merged_df.to_csv(output_file)
        print(f"Merged CSV file saved to: {output_file}")


root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "EF_single_functions.csv"
output_csv_file = "merged_output_DUD-E.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)


In [ ]:
import os
import pandas as pd

def rename_and_merge_csv_files(root_dir, target_file, output_file):
    merged_df = None

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_file:
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, index_col=0)
                dirname = os.path.basename(dirpath)
                df.rename(columns={"EF1%": dirpath.replace('/media/mario/T7/FINISHED/FINISHED/', '').replace('/temp/consensus', '')}, inplace=True)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["clustering_method", "selected_columns", 'method_name'])
                    print(merged_df.head())

    if merged_df is not None:
        # Adding the average column
        columns_to_exclude = ['method_name', 'selected_columns', 'clustering_metric']
        numeric_columns = [col for col in merged_df.columns if col not in columns_to_exclude]
        merged_df['Average'] = merged_df[numeric_columns].mean(axis=1)
        print(merged_df.head())
        merged_df.to_csv(output_file)
        print(f"Merged CSV file saved to: {output_file}")

root_directory = "/media/mario/T7/FINISHED/FINISHED/"
target_csv_file = "consensus_summary.csv"
output_csv_file = "merged_output_consensus_DUD-E.csv"

rename_and_merge_csv_files(root_directory, target_csv_file, output_csv_file)
